In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('playground').setMaster('local')
sc = SparkContext.getOrCreate(conf=conf)
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 10:54:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=lecture-lyon2>

In [2]:
from pyspark.rdd import RDD

import pandas as pd
%matplotlib inline

In [3]:
def ungroup_input(sc, system):
    """
    Generate the websystem as an RDD of tuples (page, neighbor page)
    """
    system = system.flatMap(lambda x: [(x[0], y) for y in x[1]])
    return system

In [4]:
"""
Graded cell

2 points
"""
system = sc.parallelize([
        ('a', ['b','c','d']),
        ('c', ['b']), 
        ('b', ['c','d']), 
        ('d', ['a','c'])
    ])
result = ungroup_input(sc, system).collect()
assert result == [
    ('a', 'b'), ('a', 'c'), ('a', 'd'), 
    ('c', 'b'), 
    ('b', 'c'), ('b', 'd'), 
    ('d', 'a'), ('d', 'c')
]

In [5]:
def group_input(sc, system):
    """
    Generate the websystem as an RDD of tuples (page, list of neighbors)
    """
    # YOUR CODE HERE
    system = system.reduceByKey(lambda x,y: x+y)
    system = system.map(lambda x: (x[0], list(set(x[1]))))
    return system

In [6]:
"""
Graded cell

2 points
"""
system = sc.parallelize([('a', 'b'), ('a', 'c'), ('a', 'd'), ('c', 'b'), ('b', 'c'), ('b', 'd'), ('d', 'a'), ('d', 'c')])
result = group_input(sc, system).collect()
print(result)
assert result == [
        ('a', ['b','c','d']),
        ('c', ['b']), 
        ('b', ['c','d']), 
        ('d', ['a','c'])
    ]

[('a', ['b', 'c', 'd']), ('c', ['b']), ('b', ['c', 'd']), ('d', ['a', 'c'])]


In [7]:
def computeContribs(urls, rank):
    """
    Calculates URL contributions to the rank of other URLs.
    """
    # YOUR CODE HERE
    contribs = [(k, rank/len(urls)) for k in urls]
    return contribs

In [8]:
"""
Graded cell

2 points
"""
assert computeContribs(['b', 'c', 'd'], 1) == [('b', 1/3), ('c', 1/3), ('d', 1/3)]

In [9]:
def generate_contributions(sc, links, ranks):
    """
    Calculates URL contributions to the rank of other URLs.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [10]:
"""
Graded cell

2 points
"""
links = sc.parallelize([
        ('a', ['b','c','d']),
        ('c', ['b']), 
        ('b', ['c','d']), 
        ('d', ['a','c'])
    ])

ranks = sc.parallelize([
        ('a', 1.0),
        ('c', 3.0), 
        ('b', 2.0), 
        ('d', 4.0)
])

assert generate_contributions(sc, links, ranks).collect() == [
    ('b', 3.0), # contribution from c
    ('c', 1.0), ('d', 1.0), # contribution from b
    ('a', 2.0), ('c', 2.0), # contribution from d
    ('b', 1/3), ('c', 1/3), ('d', 1/3) # contribution from a 
]

NotImplementedError: 